In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
from torchvision.datasets.utils import download_url
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

In [ ]:
import torch
import tarfile
from torchvision.datasets.utils import download_url
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import csv
from PIL import Image

In [ ]:
data = open(f"file directory")  #enter dataset directory here

In [ ]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
for x in digit:
    print(x, ": ", len(os.listdir('/content/data/'+x)))
#this is for balancing and for informations about our dataset

In [ ]:
import csv
d = {}
with open("Spoken_digit.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["File", "Label"])
    for x in digit:
        if os.path.isdir('/content/data/'+x):
            d[x] = os.listdir('/content/data/'+x)
            for name in os.listdir('/content/data/'+x):
                if os.path.isfile('/content/data/'+x+"/"+name):
                    csvwriter.writerow([x+'/'+name, x])

#shuffle 
df = pd.read_csv('Spoken_digit.csv')
df = df.sample(frac=1)
df.to_csv('Spoken_digit.csv', index = False)

dfx = pd.DataFrame.from_dict(d, orient='index', dtype = 'float32').transpose()
dfx.to_csv('Spoken_digit_X.csv', index = False)
'''

In [ ]:
#main part for feature extraction

def extract_features(path):
    data, sr = librosa.load('/content/data/'+path)
    mfccs = np.mean(librosa.feature.mfcc(y = data, sr=sr).T, axis = 0)
    spectral_centroids = librosa.feature.spectral_centroid(data+0.01, sr=sr)[0]
    stft = np.abs(librosa.stft(data))
    chroma = np.mean(librosa.feature.chroma_stft(S = stft, sr = sr).T, axis = 0)
    mel = np.mean(librosa.feature.melspectrogram(data, sr).T, axis = 0)
    contrast = np.mean(librosa.feature.spectral_contrast(S = stft, sr = sr).T, axis = 0)
    tonnetz = np.mean(librosa.feature.tonnetz(y = librosa.effects.harmonic(data), sr = sr).T, axis = 0)
    
    #print(mfccs.shape, spectral_centroids.shape, stft.shape, chroma.shape, mel.shape, contrast.shape, tonnetz.shape)
    
    #spectral_centroids have varying shapes for each datapoint and stft is 2d array. Thus they are not included in the final features.
    return np.concatenate((mfccs, chroma, mel, contrast, tonnetz), axis = 0).astype('float32')

In [ ]:
class SpokenDigist(Dataset):
    def __init__(self, file = None, rootdir = None):
        self.df = pd.read_csv(file)
        self.rootdir = rootdir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.loc[i]
        fname, label = row['File'], row['Label']
        fts = extract_features(fname)
        #print(len(fts))
        return torch.tensor(fts), torch.tensor(digit.index(label))
    
    def getsr(self, i):
        fname, label = row['File'], row['Label']
        _, sr = librosa.load(self.rootdir+'/'+fname)
        return sr
    
#Labeling features

In [ ]:
spoken_dset = SpokenDigit(file = "Spoken_digit.csv", rootdir = "/content/data/")

In [ ]:
#after we extracted features we must put features into our model to train.

In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

In [ ]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

In [ ]:
# Spoken_mnist_finalfts.csv is generated in last part(feature extraction)

In [ ]:
finalfts = pd.read_csv("Spoken_digit_finalfts.csv")

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
finalfts[finalfts.columns[1:]] = scale.fit_transform(finalfts[finalfts.columns[1:]])

spokendset = TensorDataset(torch.tensor(np.array(finalfts[finalfts.columns[1:]].astype('float32'))),torch.tensor(finalfts['Label'])) 

# 90-10 split
size = len(spokendset)
val_size = int(0.1 * size)
train_size = size - val_size 

train_dset, val_dset = random_split(spokendset, [train_size, val_size])

train_size, val_size

train_dl = DataLoader(train_dset, 512, True)
val_dl = DataLoader(val_dset, 512)

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [ ]:
#train part************************************************************

class SpokenDigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(173, 1024)
        self.l2 = nn.Linear(1024, 512)
        self.l3 = nn.Linear(512, 64)
        self.l4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x

    def training_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        _, pred = torch.max(outputs, 1)
        accuracy = torch.tensor(torch.sum(pred==labels).item()/len(pred))
        return [loss.detach(), accuracy.detach()] 

In [ ]:
def evaluate(model, loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in loader]
    outputs = torch.tensor(outputs).T
    loss, accuracy = torch.mean(outputs, dim=1)
    return {"loss" : loss.item(), "accuracy" : accuracy.item()}

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
def fit(model, train_loader, val_loader, epochs, lr, optimizer_function = torch.optim.Adam):
    history = []
    optimizer = optimizer_function(model.parameters(), lr)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        print("Epoch ", epoch)
        #Train
        model.train()
        lrs = []
        tr_loss = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            tr_loss.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        #Validate
        result = evaluate(model, val_loader)
        result["lrs"] = lrs
        result["train loss"] = torch.stack(tr_loss).mean().item()
 
        print("Last lr: ", lrs[-1]," Train_loss: ", result["train loss"], " Val_loss: ", result['loss'], " Accuracy: ", result['accuracy'])
        history.append(result)         
    return history

In [ ]:
model = to_device(SpokenDigitModel(), device)
history = []
evaluate(model, val_dl)

In [ ]:
history.append(fit(model, train_dl, val_dl, 64, 0.01))

In [ ]:
@torch.no_grad()
def predict_dl(model, dl):
    torch.cuda.empty_cache()
    batch_probs = []
    batch_targ = []
    for xb, yb in dl:
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
        batch_targ.append(yb.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    batch_targ = torch.cat(batch_targ)
    return [list(values).index(max(values)) for values in batch_probs], batch_targ

In [ ]:
r = evaluate(model, val_dl)
yp, yt = predict_dl(model, val_dl)
print("Loss: ", r['loss'], "\nAccuracy: ", r['accuracy'], "\nF-score: ", f1_score(yt, yp, average='micro'))